In [ ]:
%pip install azure.storage.blob

In [ ]:
import io
import numpy as np
import pandas as pd
from azure.storage.blob import BlobServiceClient


blob_service_client = BlobServiceClient.from_connection_string(
    "CONN_STR"
)

In [ ]:
blob_client = blob_service_client.get_blob_client("raw", "df_mg.csv")
df_mg = pd.read_csv(io.StringIO(blob_client.download_blob().content_as_text()))
df_mg

In [ ]:
blob_client = blob_service_client.get_blob_client("raw", "df_total.csv")
df_total = pd.read_csv(io.StringIO(blob_client.download_blob().content_as_text()))
df_total

In [ ]:
df_mg = df_mg[df_mg["type"] == "/subscriptions"]
df_mg["id"] = df_mg["id"].str.split("/").str.get(2)
df_mg["mg0"] = (
    df_mg["properties.parentDisplayNameChain"]
    .str.split(", '")
    .str.get(-1)
    .str.strip("'[]")
)
df_mg[["Management", "mg2", "Department", "mg4"]] = (
    df_mg["mg0"].astype(str).str.split("_", expand=True)
)
df_mg = df_mg[df_mg["Management"].isin(["Cenitex", "Customer", "Tier"])]
df_mg["Department"].replace(
    {np.nan: "Cenitex", "Dev": "Cenitex"}, inplace=True
)
df_mg["Management"].replace(
    {
        "Cenitex": "Cenitex Managed",
        "Customer": "Customer Managed",
        "Tier": "T0",
    },
    inplace=True,
)
df_mg = df_mg[["id", "properties.displayName", "Management", "Department"]]
df_mg.columns = ["id", "subscriptionName", "management", "department"]
df_mg.reset_index(drop=True, inplace=True)
df_mg

In [ ]:
df_total["id"] = df_total["id"].str.split("/").str.get(2)
df = pd.merge(
    left=df_total,
    right=df_mg,
    left_on="id",
    right_on="id",
    how="left",
    indicator=False,
)
df.replace(
    {"australiasoutheast": "Melbourne", "australiaeast": "Sydney"}, inplace=True
)
df["vCores"] = (
    df["properties.hardwareProfile.vmSize"]
    .apply(
        lambda cores: "".join(c for c in cores.split("_")[1] if c.isnumeric())
    )
    .astype(int)
)
df = df[
    [
        "name",
        "subscriptionName",
        "management",
        "department",
        "location",
        "properties.storageProfile.osDisk.osType",
        "properties.hardwareProfile.vmSize",
        "vCores",
        "properties.licenseType",
        "properties.storageProfile.osDisk.createOption",
        "properties.storageProfile.imageReference.publisher",
        "properties.storageProfile.imageReference.offer",
        "plan.name",
    ]
]
df.columns = [
    "Name",
    "Subscription",
    "management",
    "department",
    "Location",
    "Operating System",
    "Size",
    "vCores",
    "Os Licensing Benefit",
    "Source",
    "Publisher",
    "Offer",
    "Plan",
]
df.reset_index(drop=True, inplace=True)
df